In [11]:
import pandas as pd
from helper.misc import flatten
import mygene

mg = mygene.MyGeneInfo()

In [12]:
repurposing_hub_drug_targets = pd.read_csv("../../data/primary_data/repurposing_hub_drug_targets.txt", sep="\t").drop(
    ['MOA', 'Phase'], axis=1).set_index('Name')
repurposing_hub_drug_targets = repurposing_hub_drug_targets[
    ~repurposing_hub_drug_targets.index.duplicated(keep='first')]
repurposing_hub_drug_targets

,Target
Name,
(R)-(-)-apomorphine,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3..."
(R)-(-)-rolipram,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A"
(R)-baclofen,"GABBR1, GABBR2"
(S)-(+)-rolipram,"PDE4B, PDE4D"
"[sar9,met(o2)11]-substance-p",TACR1
...,...
8-M-PDOT,"MTNR1A, MTNR1B"
80841-78-7,NaN
9-aminoacridine,NaN


In [29]:
repurposing_hub_drugs = pd.read_csv('../../data/primary_data/repurposing_hub_drugs.txt', skiprows=9, sep='\t')
repurposing_hub_drugs['broad_id'] = repurposing_hub_drugs['broad_id'].apply(lambda x: x[:-9])
repurposing_hub_drugs = repurposing_hub_drugs.loc[:, ['broad_id', 'pert_iname', 'InChIKey']].drop_duplicates().rename(
    {'pert_iname': 'Name'}, axis=1).set_index('Name')
repurposing_hub_drugs = repurposing_hub_drugs[~repurposing_hub_drugs.index.duplicated(keep='first')]
repurposing_hub_drugs

,broad_id,InChIKey
Name,,
(R)-(-)-apomorphine,BRD-K76022557,VMWNQDUVQKEIOC-CYBMUJFWSA-N
(R)-(-)-rolipram,BRD-K75516118,HJORMJIFDVBMOB-LBPRGKRZSA-N
(R)-baclofen,BRD-K62353271,KPYSYYIEGFHWSV-QMMMGPOBSA-N
(S)-(+)-rolipram,BRD-K65856711,HJORMJIFDVBMOB-GFCCVEGCSA-N
"[sar9,met(o2)11]-substance-p",BRD-K89787693,OUPXSLGGCPUZJJ-SARDKLJWSA-N
...,...,...
zoxazolamine,BRD-K66353228,YGCODSQDUUUKIV-UHFFFAOYSA-N
ZSET1446,BRD-K35984734,QZWYXEBIQWJXAR-UHFFFAOYSA-N
ZSTK-474,BRD-K63068307,HGVNLRPZOWWDKD-UHFFFAOYSA-N


In [30]:
drh = pd.concat([repurposing_hub_drug_targets, repurposing_hub_drugs], axis=1).set_index('InChIKey')
drh = drh[~drh.index.duplicated(keep='first')]
drh

,Target,broad_id
InChIKey,,
VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...",BRD-K76022557
HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A",BRD-K75516118
KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",BRD-K62353271
HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D",BRD-K65856711
OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,BRD-K89787693
...,...,...
RVIGBTUDFAGRTQ-NSHDSACASA-N,"MTNR1A, MTNR1B",BRD-A07232941
QCLFSYYUWPUWQR-UHFFFAOYSA-N,NaN,BRD-K03044000
XJGFWWJLMVZSIG-UHFFFAOYSA-N,NaN,BRD-K00535541


In [42]:
bdb = pd.read_csv('BindingDB_All_2022m4.tsv.zip', compression='zip', sep="\t",
                  usecols=['Ligand InChI Key', 'Target Source Organism According to Curator or DataSource',
                           'UniProt (SwissProt) Primary ID of Target Chain'], low_memory=False)
bdb = bdb[bdb['Target Source Organism According to Curator or DataSource'] == 'Homo sapiens'].drop(
    'Target Source Organism According to Curator or DataSource', axis=1).dropna()
bdb = bdb.groupby('Ligand InChI Key')['UniProt (SwissProt) Primary ID of Target Chain'].apply(
    lambda x: ', '.join(x)).to_frame()

uniprots = list(
    set(flatten(bdb.loc[:, 'UniProt (SwissProt) Primary ID of Target Chain'].apply(lambda x: str(x).split(', ')))))
uniprot_gene_mapping = mg.querymany(uniprots, scopes='uniprot', fields='symbol', species='human',
                                    as_dataframe=True).loc[:, 'symbol'].dropna()
uniprot_gene_mapping

querying 1-1000...done.
querying 1001-1787...done.
Finished.
12 input query terms found dup hits:
	[('P0C869', 2), ('P31941', 2), ('Q96BR1', 2), ('P36544', 2), ('P47929', 2), ('P57059', 2), ('Q9NQU5'
19 input query terms found no hit:
	['P29477', 'Q92753', 'P0DUB6,P0DTE7,P0DTE8', 'P42227', 'P19835', 'P0DMV8,P0DMV9', 'P61169', 'P29476'
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


query
Q8NE63      HIPK4
Q6P1M0    SLC27A4
P14384        CPM
Q8N4C6        NIN
Q04771      ACVR1
           ...   
P51575      P2RX1
Q16739       UGCG
Q9Y6R4     MAP3K4
P30556      AGTR1
P48443       RXRG
Name: symbol, Length: 1780, dtype: object

In [57]:
uniprots = list(
    set(flatten(bdb.loc[:, 'UniProt (SwissProt) Primary ID of Target Chain'].apply(lambda x: str(x).split(', ')))))
len(uniprots)

1787

In [59]:
bdb

,UniProt (SwissProt) Primary ID of Target Chain,BDB Target
Ligand InChI Key,,
AAAAEENPAALFRN-UHFFFAOYSA-N,Q9UBL9,[P2RX2]
AAAAZQPHATYWOK-JXMROGBWSA-N,P00533,[EGFR]
AAACBXVBBDAYRQ-UHFFFAOYSA-N,Q5S007,[LRRK2]
AAADCKOTKJSVMJ-SZNDQCEHSA-N,P56524,[HDAC4]
AAADPBLPXCELKR-UHFFFAOYSA-N,P20309,[CHRM3]
...,...,...
ZZZWVADOTWQQFE-VWLOTQADSA-N,P00742,[F10]
ZZZWVMJSEFVIAC-UHFFFAOYSA-N,"Q13535, Q13535",[ATR]
ZZZYFYOJEILVNN-UHFFFAOYSA-N,P08246,[ELANE]


In [67]:
import numpy as np


def map_uniprot_to_gene(x):
    uniprot_genes = list(map(str.strip, x.split(",")))
    gene_names = []
    for x in uniprot_genes:
        if x in uniprot_gene_mapping.index:
            mapping = uniprot_gene_mapping.loc[x]
            if type(mapping) == pd.Series:
                gene_names.append(mapping.values[0])
            else:
                gene_names.append(mapping)

    return ", ".join(gene_names)


bdb['BDB Target'] = bdb['UniProt (SwissProt) Primary ID of Target Chain'].apply(
    lambda x: map_uniprot_to_gene(str(x)))


def list_to_string(x):
    try:
        str_list = ", ".join(x)
    except TypeError:
        ls = [str(xs) for xs in x]
        str_list = ", ".join(ls)
    return str_list



In [68]:
bdb

,UniProt (SwissProt) Primary ID of Target Chain,BDB Target
Ligand InChI Key,,
AAAAEENPAALFRN-UHFFFAOYSA-N,Q9UBL9,P2RX2
AAAAZQPHATYWOK-JXMROGBWSA-N,P00533,EGFR
AAACBXVBBDAYRQ-UHFFFAOYSA-N,Q5S007,LRRK2
AAADCKOTKJSVMJ-SZNDQCEHSA-N,P56524,HDAC4
AAADPBLPXCELKR-UHFFFAOYSA-N,P20309,CHRM3
...,...,...
ZZZWVADOTWQQFE-VWLOTQADSA-N,P00742,F10
ZZZWVMJSEFVIAC-UHFFFAOYSA-N,"Q13535, Q13535","ATR, ATR"
ZZZYFYOJEILVNN-UHFFFAOYSA-N,P08246,ELANE


In [79]:
pd.concat([drh, bdb], axis=1, join='outer').dropna(subset='broad_id').drop(
    'UniProt (SwissProt) Primary ID of Target Chain', axis=1).drop_duplicates().reset_index().set_index('broad_id').rename({'index': 'InChIKey'}, axis=1).to_csv('bdb_drh_merge.csv', index=True)